In [11]:
#!pip freeze
#!pip install numpy
#!pip install pandas
#!pip install matplotlib


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
print('My first gitHub from VSCode')

My first gitHub from VSCode
